In [ ]:
#| default_exp docments

# Docments

> Document parameters using comments.

In [ ]:
#| export
import re,ast,inspect
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef,AsyncFunctionDef,AnnAssign
from io import BytesIO
from textwrap import dedent
from types import SimpleNamespace
from inspect import getsource,isfunction,ismethod,isclass,signature,Parameter
from dataclasses import dataclass, is_dataclass
from fastcore.utils import *
from fastcore.meta import delegates
from fastcore import docscrape
from textwrap import fill
from inspect import isclass,getdoc,signature

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

`docments` provides programmatic access to comments in function parameters and return types. It can be used to create more developer-friendly documentation, CLI, etc tools.

## Why?

Without docments, if you want to document your parameters, you have to repeat param names in docstrings, since they're already in the function signature. The parameters have to be kept synchronized in the two places as you change your code. Readers of your code have to look back and forth between two places to understand what's happening. So it's more work for you, and for your users.

Furthermore, to have parameter documentation formatted nicely without docments, you have to use special magic docstring formatting, often with [odd quirks](https://stackoverflow.com/questions/62167540/why-do-definitions-have-a-space-before-the-colon-in-numpy-docstring-sections), which is a pain to create and maintain, and awkward to read in code. For instance, using [numpy-style documentation](https://numpydoc.readthedocs.io/en/latest/format.html):

In [ ]:
def add_np(a:int, b:int=0)->int:
    """The sum of two numbers.
    
    Used to demonstrate numpy-style docstrings.

Parameters
----------
a : int
    the 1st number to add
b : int
    the 2nd number to add (default: 0)

Returns
-------
int
    the result of adding `a` to `b`"""
    return a+b

By comparison, here's the same thing using docments:

In [ ]:
def add(
    a:int, # the 1st number to add
    b=0,   # the 2nd number to add
)->int:    # the result of adding `a` to `b`
    "The sum of two numbers."
    return a+b

## Numpy docstring helper functions

`docments` also supports numpy-style docstrings, or a mix or numpy-style and docments parameter documentation. The functions in this section help get and parse this information.

In [ ]:
#| export
def docstring(sym):
    "Get docstring for `sym` for functions ad classes"
    if isinstance(sym, str): return sym
    res = getdoc(sym)
    if not res and isclass(sym): res = getdoc(sym.__init__)
    return res or ""

In [ ]:
test_eq(docstring(add), "The sum of two numbers.")

In [ ]:
#| export
def parse_docstring(sym):
    "Parse a numpy-style docstring in `sym`"
    return AttrDict(**docscrape.NumpyDocString(docstring(sym)))

In [ ]:
# parse_docstring(add_np)

In [ ]:
#| export
def isdataclass(s):
    "Check if `s` is a dataclass but not a dataclass' instance"
    return is_dataclass(s) and isclass(s)

In [ ]:
#| export
def get_dataclass_source(s):
    "Get source code for dataclass `s`"
    return getsource(s) if not getattr(s, "__module__") == '__main__' else ""

In [ ]:
#| export
def get_source(s):
    "Get source code for string, function object or dataclass `s`"
    if isinstance(s,str): return s
    return getsource(s) if isfunction(s) or ismethod(s) else get_dataclass_source(s) if isdataclass(s) else None

In [ ]:
#| export
def _parses(s):
    "Parse Python code in string, function object or dataclass `s`"
    return parse(dedent(get_source(s) or ''))

def _tokens(s):
    "Tokenize Python code in string or function object `s`"
    s = get_source(s)
    if not s: return []
    return tokenize(BytesIO(s.encode('utf-8')).readline)

_clean_re = re.compile(r'^\s*#(.*)\s*$')
def _clean_comment(s):
    res = _clean_re.findall(s)
    return res[0] if res else None

def _param_locs(s, returns=True, args_kwargs=False):
    "`dict` of parameter line numbers to names"
    body = _parses(s).body
    if len(body)==1:
        defn = body[0]
        if isinstance(defn, (FunctionDef, AsyncFunctionDef)):
            res = {arg.lineno:arg.arg for arg in defn.args.args}
            # Add *args if present
            if defn.args.vararg: res[defn.args.vararg.lineno] = defn.args.vararg.arg
            # Add keyword-only args
            res.update({arg.lineno:arg.arg for arg in defn.args.kwonlyargs})
            # Add **kwargs if present
            if defn.args.kwarg and args_kwargs: res[defn.args.kwarg.lineno] = defn.args.kwarg.arg
            if returns and defn.returns: res[defn.returns.lineno] = 'return'
            return res
        elif isdataclass(s):
            res = {arg.lineno:arg.target.id for arg in defn.body if isinstance(arg, AnnAssign)}
            return res
    return None

In [ ]:
parms = _param_locs(add)
parms

{2: 'a', 3: 'b', 4: 'return'}

In [ ]:
#| export
empty = Parameter.empty

In [ ]:
#| export
def _get_comment(line, arg, comments, parms):
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

def _get_full(p, docs, eval_str=False):
    anno = p.annotation
    if anno==empty:
        if p.default!=empty: anno = type(p.default)
        elif p.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD): anno = p.kind
        elif eval_str: anno = None
    return AttrDict(docment=docs.get(p.name), anno=anno, default=p.default)

In [ ]:
_get_comment(2, 'a', {2: ' the 1st number to add'}, parms)

'the 1st number to add'

In [ ]:
#| export
def _merge_doc(dm, npdoc):
    if not npdoc: return dm
    if not isinstance(dm, dict): return dm or '\n'.join(npdoc.desc)
    # if not dm.anno or dm.anno==empty: dm.anno = npdoc.type
    if not dm.docment: dm.docment = '\n'.join(npdoc.desc)
    return dm

def _merge_docs(dms, npdocs):
    npparams = npdocs['Parameters']
    params = {nm:_merge_doc(dm,npparams.get(nm,None)) for nm,dm in dms.items()}
    if 'return' in dms: params['return'] = _merge_doc(dms['return'], npdocs['Returns'])
    return params

In [ ]:
#| export
def _get_property_name(p):
    "Get the name of property `p`"
    if hasattr(p, 'fget'):
        return p.fget.func.__qualname__ if hasattr(p.fget, 'func') else p.fget.__qualname__
    else: return next(iter(re.findall(r'\'(.*)\'', str(p)))).split('.')[-1]

In [ ]:
#| export
def get_name(obj):
    "Get the name of `obj`"
    if isinstance(obj, partial):
        nm = get_name(obj.func)
        args = [repr(a) for a in obj.args] + [f'{k}={repr(v)}' for k,v in obj.keywords.items()]
        return f"{nm}[partial: {', '.join(args)}]"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1]
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [ ]:
test_eq(get_name(in_ipython), 'in_ipython')
test_eq(get_name(L.map), 'map')

In [ ]:
#| export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [ ]:
assert qual_name(docscrape) == 'fastcore.docscrape'

## Docments

Let's manually go through each step of `_docments` to see what it does:

In [ ]:
def _b(
    z:str='b', # Last
):
    return b, a

@delegates(_b)
def _c(
    b:str, # Ignore
    a:int=2
): return b, a # Third

@delegates(_c)
def _d(
    c:int, # First
    b:str, # Second
    **kwargs
)->int: # Return an int
    return c, _c(b, **kwargs)

In [ ]:
s = _d
nps = parse_docstring(s)
if isclass(s) and not is_dataclass(s): s = s.__init__
comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
comments

{3: ' First', 4: ' Second', 6: ' Return an int'}

In [ ]:
parms = _param_locs(s, returns=True, args_kwargs=True) or {}
parms

{3: 'c', 4: 'b', 5: 'kwargs', 6: 'return'}

In [ ]:
docs = {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}
docs

{'c': 'First', 'b': 'Second', 'kwargs': None, 'return': 'Return an int'}

In [ ]:
sig = signature(s, eval_str=True)
res = {name:_get_full(p, docs) for name,p in sig.parameters.items()}
res

{'c': {'docment': 'First', 'anno': int, 'default': inspect._empty},
 'b': {'docment': 'Second', 'anno': str, 'default': inspect._empty},
 'a': {'docment': None, 'anno': int, 'default': 2},
 'z': {'docment': None, 'anno': str, 'default': 'b'}}

In [ ]:
res['return'] = AttrDict(docment=docs.get('return'), anno=sig.return_annotation, default=empty)
res = _merge_docs(res, nps)
res

{'c': {'docment': 'First', 'anno': int, 'default': inspect._empty},
 'b': {'docment': 'Second', 'anno': str, 'default': inspect._empty},
 'a': {'docment': None, 'anno': int, 'default': 2},
 'z': {'docment': None, 'anno': str, 'default': 'b'},
 'return': {'docment': 'Return an int',
  'anno': int,
  'default': inspect._empty}}

In [ ]:
_d.__delwrap__

<function __main__._c(b: str, a: int = 2, *, z: str = 'b')>

In [ ]:
#| export
def docments(s, full=False, eval_str=False, returns=True, args_kwargs=False):
    "Get docments for `s`"
    if isclass(s) and not is_dataclass(s): s = s.__init__
    try: sig = signature_ex(s, eval_str=eval_str)
    except ValueError: return AttrDict()
    nps = parse_docstring(s)
    docs = {}
    while s:
        p = _param_locs(s, returns=returns, args_kwargs=args_kwargs) or {}
        c = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
        for k,v in p.items():
            if v not in docs: docs[v] = _get_comment(k, v, c, p)
        s = getattr(s, '__delwrap__', None)
    
    res = {k:_get_full(v, docs, eval_str=eval_str) if full else docs.get(k) for k,v in sig.parameters.items()}
    if returns:
        if full: res['return'] = AttrDict(docment=docs.get('return'), anno=sig.return_annotation, default=empty)
        else: res['return'] = docs.get('return')
    return AttrDict(_merge_docs(res, nps))

In [ ]:
docments(_d)

```python
{'a': None, 'b': 'Second', 'c': 'First', 'return': 'Return an int', 'z': 'Last'}
```

In [ ]:
docments(_d, full=True)

```python
{ 'a': {'anno': <class 'int'>, 'default': 2, 'docment': None},
  'b': { 'anno': <class 'str'>,
         'default': <class 'inspect._empty'>,
         'docment': 'Second'},
  'c': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'First'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'Return an int'},
  'z': {'anno': <class 'str'>, 'default': 'b', 'docment': 'Last'}}
```

The returned `dict` has parameter names as keys, docments as values. The return value comment appears in the `return`, unless `returns=False`. Using the `add` definition above, we get:

In [ ]:
def add(
    a:int, # the 1st number to add
    b=0,   # the 2nd number to add
)->int:    # the result of adding `a` to `b`
    "The sum of two numbers."
    return a+b

In [ ]:
docments(add)

```python
{ 'a': 'the 1st number to add',
  'b': 'the 2nd number to add',
  'return': 'the result of adding `a` to `b`'}
```

`args_kwargs=True` adds args and kwargs docs too:

In [ ]:
def add(
    a:int, # the 1st number to add
    *args, # some args
    b=0,   # the 2nd number to add
    **kwargs, # Passed to the `example` function
)->int:    # the result of adding `a` to `b`
    "The sum of two numbers."
    return a+b

In [ ]:
docments(add, args_kwargs=True)

```python
{ 'a': 'the 1st number to add',
  'args': 'some args',
  'b': 'the 2nd number to add',
  'kwargs': 'Passed to the `example` function',
  'return': 'the result of adding `a` to `b`'}
```

If you pass `full=True`, the values are `dict` of defaults, types, and docments as values. Note that the type annotation is inferred from the default value, if the annotation is empty and a default is supplied. (Note that for `full`, `args_kwargs=True` is always set too.)

In [ ]:
docments(add, full=True)

```python
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'the 1st number to add'},
  'args': { 'anno': <_ParameterKind.VAR_POSITIONAL: 2>,
            'default': <class 'inspect._empty'>,
            'docment': 'some args'},
  'b': { 'anno': <class 'int'>,
         'default': 0,
         'docment': 'the 2nd number to add'},
  'kwargs': { 'anno': <_ParameterKind.VAR_KEYWORD: 4>,
              'default': <class 'inspect._empty'>,
              'docment': None},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'the result of adding `a` to `b`'}}
```

To evaluate stringified annotations (from python 3.10), use `eval_str`:

In [ ]:
docments(add, full=True, eval_str=True)['a']

```python
{ 'anno': <class 'int'>,
  'default': <class 'inspect._empty'>,
  'docment': 'the 1st number to add'}
```

In [ ]:
docments(add, full=True)['a']

```python
{ 'anno': <class 'int'>,
  'default': <class 'inspect._empty'>,
  'docment': 'the 1st number to add'}
```

If you need more space to document a parameter, place one or more lines of comments above the parameter, or above the return type. You can mix-and-match these docment styles:

In [ ]:
def add(
    # The first operand
    a:int,
    # This is the second of the operands to the *addition* operator.
    # Note that passing a negative value here is the equivalent of the *subtraction* operator.
    b:int,
)->int: # The result is calculated using Python's builtin `+` operator.
    "Add `a` to `b`"
    return a+b

In [ ]:
docments(add)

```python
{ 'a': 'The first operand',
  'b': 'This is the second of the operands to the *addition* operator.\n'
       'Note that passing a negative value here is the equivalent of the '
       '*subtraction* operator.',
  'return': "The result is calculated using Python's builtin `+` operator."}
```

Docments works with async functions, too:

In [ ]:
async def add_async(
    # The first operand
    a:int,
    # This is the second of the operands to the *addition* operator.
    # Note that passing a negative value here is the equivalent of the *subtraction* operator.
    b:int,
)->int: # The result is calculated using Python's builtin `+` operator.
    "Add `a` to `b`"
    return a+b

In [ ]:
test_eq(docments(add_async), docments(add))

You can also use docments with classes and methods:

In [ ]:
class Adder:
    "An addition calculator"
    def __init__(self,
        a:int, # First operand
        b:int, # 2nd operand
    ): self.a,self.b = a,b
    
    def calculate(self
                 )->int: # Integral result of addition operator
        "Add `a` to `b`"
        return a+b

In [ ]:
docments(Adder)

```python
{'a': 'First operand', 'b': '2nd operand', 'return': None, 'self': None}
```

In [ ]:
docments(Adder.calculate)

```python
{'return': 'Integral result of addition operator', 'self': None}
```

docments can also be extracted from numpy-style docstrings:

In [ ]:
print(add_np.__doc__)

The sum of two numbers.

    Used to demonstrate numpy-style docstrings.

Parameters
----------
a : int
    the 1st number to add
b : int
    the 2nd number to add (default: 0)

Returns
-------
int
    the result of adding `a` to `b`


In [ ]:
docments(add_np)

```python
{ 'a': 'the 1st number to add',
  'b': 'the 2nd number to add (default: 0)',
  'return': 'the result of adding `a` to `b`'}
```

You can even mix and match docments and numpy parameters:

In [ ]:
def add_mixed(a:int, # the first number to add
              b
             )->int: # the result
    """The sum of two numbers.

Parameters
----------
b : int
    the 2nd number to add (default: 0)"""
    return a+b

In [ ]:
docments(add_mixed, full=True)

```python
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'the first number to add'},
  'b': { 'anno': <class 'inspect._empty'>,
         'default': <class 'inspect._empty'>,
         'docment': 'the 2nd number to add (default: 0)'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'the result'}}
```

You can use docments with dataclasses, however if the class was defined in online notebook, docments will not contain parameters' comments. This is because the source code is not available in the notebook. After converting the notebook to a module, the docments will be available. Thus, documentation will have correct parameters' comments.

In [ ]:
#| hide
class _F:
    @classmethod
    def class_method(cls, 
                     foo:str, # docment for parameter foo
                     ):...
    
test_eq(docments(_F.class_method), {'foo': 'docment for parameter foo', 'return': None})

Docments even works with `delegates`:

In [ ]:
from fastcore.meta import delegates

In [ ]:
def _a(a:str=None): return a # First

@delegates(_a)
def _b(b:str, # Second
       **kwargs
      ): # Return nothing
    return b, (_a(**kwargs)) 

docments(_b)

```python
{'a': 'First', 'b': 'Second', 'return': None}
```

In [ ]:
docments(_b, full=True)

```python
{ 'a': {'anno': <class 'str'>, 'default': None, 'docment': 'First'},
  'b': { 'anno': <class 'str'>,
         'default': <class 'inspect._empty'>,
         'docment': 'Second'},
  'return': { 'anno': <class 'inspect._empty'>,
              'default': <class 'inspect._empty'>,
              'docment': None}}
```

Builtins just return an empty dict:

In [ ]:
docments(str)

```python
{'args': None, 'kwargs': None, 'return': None, 'self': None}
```

## Extract docstrings

In [ ]:
#| export
def sig_source(obj):
    "Full source of signature line(s) for a function or class."
    src = inspect.getsource(obj)
    tree = ast.parse(src)
    body_start = tree.body[0].body[0].lineno
    if body_start == 1: return src.splitlines()[0]
    return '\n'.join(src.splitlines()[:body_start-1])

In [ ]:
print(sig_source(flexiclass))

def flexiclass(
        cls # The class to convert
    ) -> dataclass:


In [ ]:
def simple(x: dict[str, int]): return x
print(sig_source(simple))

def simple(x: dict[str, int]): return x


In [ ]:
def multi(a, b=1,
          c=2,
          d=3):
    return a
print(sig_source(multi))

def multi(a, b=1,
          c=2,
          d=3):


In [ ]:
#| export
def _get_params(node):
    params = [a.arg for a in node.args.args]
    if node.args.vararg: params.append(f"*{node.args.vararg.arg}")
    if node.args.kwarg: params.append(f"**{node.args.kwarg.arg}")
    return ", ".join(params)

In [ ]:
#| export
class _DocstringExtractor(ast.NodeVisitor):
    def __init__(self): self.docstrings,self.cls,self.cls_init = {},None,None

    def visit_FunctionDef(self, node):
        name = node.name
        if name == '__init__':
            self.cls_init = node
            return
        elif name.startswith('_'): return
        elif self.cls: name = f"{self.cls}.{node.name}"
        docs = ast.get_docstring(node)
        params = _get_params(node)
        if docs: self.docstrings[name] = (docs, params)
        self.generic_visit(node)

    def visit_ClassDef(self, node):
        self.cls,self.cls_init = node.name,None
        docs = ast.get_docstring(node)
        if docs: self.docstrings[node.name] = ()
        self.generic_visit(node)
        if not docs and self.cls_init: docs = ast.get_docstring(self.cls_init)
        params = _get_params(self.cls_init) if self.cls_init else ""
        if docs: self.docstrings[node.name] = (docs, params)
        self.cls,self.cls_init = None,None

    def visit_Module(self, node):
        module_doc = ast.get_docstring(node)
        if module_doc: self.docstrings['_module'] = (module_doc, "")
        self.generic_visit(node)

In [ ]:
#| export
def extract_docstrings(code):
    "Create a dict from function/class/method names to tuples of docstrings and param lists"
    extractor = _DocstringExtractor()
    extractor.visit(ast.parse(code))
    return extractor.docstrings

In [ ]:
sample_code = """
"This is a module."

def top_func(a, b, *args, **kw):
    "This is top-level."
    pass

class SampleClass:
    "This is a class."

    def __init__(self, x, y):
        "Constructor for SampleClass."
        pass

    def method1(self, param1):
        "This is method1."
        pass

    def _private_method(self):
        "This should not be included."
        pass

class AnotherClass:
    def __init__(self, a, b):
        "This class has no separate docstring."
        pass"""

exp = {'_module': ('This is a module.', ''),
       'top_func': ('This is top-level.', 'a, b, *args, **kw'),
       'SampleClass': ('This is a class.', 'self, x, y'),
       'SampleClass.method1': ('This is method1.', 'self, param1'),
       'AnotherClass': ('This class has no separate docstring.', 'self, a, b')}
test_eq(extract_docstrings(sample_code), exp)

## Rendering docment Tables

Render nicely formatted tables that shows `docments` for any function or method.  

In [ ]:
#| export
def _non_empty_keys(d:dict): return L([k for k,v in d.items() if v != inspect._empty])
def _bold(s): return f'**{s}**' if s.strip() else s

In [ ]:
#| export
def _escape_markdown(s):
    for c in '|^': s = re.sub(rf'\\?\{c}', rf'\{c}', s)
    return s.replace('\n', '<br>')

In [ ]:
#| hide
test_eq(_escape_markdown('|'), '\|')
test_eq(_escape_markdown('\|'), '\|')
test_eq(_escape_markdown(' ^[_'), ' \^[_') # footnotes
test_eq(_escape_markdown('foo ^[_'), 'foo \^[_')
test_eq(_escape_markdown(' \^[_'), ' \^[_') #if it is already escaped leave it alone
test_eq(_escape_markdown('a long\nsentence'), 'a long<br>sentence')

In [ ]:
#| export
def _maybe_nm(o):
    if (o == inspect._empty): return ''
    else: return o.__name__ if hasattr(o, '__name__') else str(o)

In [ ]:
#| hide
test_eq(_maybe_nm(list), 'list')
test_eq(_maybe_nm('fastai'), 'fastai')

In [ ]:
#| export
def _list2row(l:list): return '| '+' | '.join([_maybe_nm(o) for o in l]) + ' |'

In [ ]:
#| hide
test_eq(_list2row(['Hamel', 'Jeremy']), '| Hamel | Jeremy |')
test_eq(_list2row([inspect._empty, bool, 'foo']), '|  | bool | foo |')

In [ ]:
#| export
class _DocmentBase:
    def __init__(self, obj):
        self.obj,self.dm = obj, docments(obj, full=True)
        if 'self' in self.dm: del self.dm['self']
    
    @property
    def has_docment(self): return any(v.get('docment') for v in self.dm.values())

In [ ]:
#| export
class DocmentTbl(_DocmentBase):
    _map = {'anno':'Type', 'default':'Default', 'docment':'Details'}

    def __init__(self, obj, verbose=True, returns=True):
        "Compute the docment table string"
        super().__init__(obj)
        self.verbose = verbose
        self.returns = False if isdataclass(obj) else returns
        try: self.params = L(signature(obj, eval_str=True).parameters.keys())
        except (ValueError,TypeError): self.params=[]
        for d in self.dm.values(): d['docment'] = ifnone(d['docment'], inspect._empty)

    @property
    def _columns(self):
        "Compute the set of fields that have at least one non-empty value so we don't show tables empty columns"
        cols = set(flatten(L(self.dm.values()).filter().map(_non_empty_keys)))
        candidates = self._map if self.verbose else {'docment': 'Details'}
        return {k:v for k,v in candidates.items() if k in cols}

    @property
    def has_docment(self): return 'docment' in self._columns and self._row_list

    @property
    def has_return(self): return self.returns and bool(_non_empty_keys(self.dm.get('return', {})))

    def _row(self, nm, props): return [nm] + [props[c] for c in self._columns]

    @property
    def _row_list(self):
        ordered_params = [(p, self.dm[p]) for p in self.params if p != 'self' and p in self.dm]
        return L([self._row(nm, props) for nm,props in ordered_params])

    @property
    def _hdr_list(self): return ['  '] + [_bold(l) for l in L(self._columns.values())]

    @property
    def hdr_str(self):
        md = _list2row(self._hdr_list)
        return md + '\n' + _list2row(['-' * len(l) for l in self._hdr_list])

    @property
    def params_str(self): return '\n'.join(self._row_list.map(_list2row))

    @property
    def return_str(self): return _list2row(['**Returns**']+[_bold(_maybe_nm(self.dm['return'][c])) for c in self._columns])

    def _repr_markdown_(self):
        if not self.has_docment: return ''
        _tbl = [self.hdr_str, self.params_str]
        if self.has_return: _tbl.append(self.return_str)
        return '\n'.join(_tbl)

    def __eq__(self,other): return self.__str__() == str(other).strip()
    __str__ = _repr_markdown_
    __repr__ = basic_repr()

`DocmentTbl` can render a markdown table showing `docments` if appropriate.  This is an example of how a `docments` table will render for a function:

In [ ]:
def _f(a,      # description of param a
       b=True, # description of param b
       c:str=None
       ) -> int: ...

_dm = DocmentTbl(_f)
_dm

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |
| **Returns** | **int** |  |  |

In [ ]:
#| hide
_exp_res="""
|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |
| **Returns** | **int** |  |  |
"""

test_eq(_dm, _exp_res)

If one column in the table has no information, for example because there are no default values, that column will not be shown.  In the below example, the **Default** column, will not be shown.  Additionally, if the return of the function is not annotated the **Returns** row will not be rendered:

In [ ]:
def _f(a,
        b:int, #param b
        c:str='foo'  #param c
       )->str: # Result of doing it
    "Do a thing"
    ...

In [ ]:
_dm2 = DocmentTbl(_f)
_dm2

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  |  |
| b | int |  | param b |
| c | str | foo | param c |
| **Returns** | **str** |  | **Result of doing it** |

`DocmentTbl` also works on classes.  By default, the `__init__` will be rendered:

In [ ]:
class _Test:
    def __init__(self,
                 a,      # description of param a
                 b=True, # description of param b
                 c:str=None):
        ...

    def foo(self,
            c:int,      # description of param c
            d=True, # description of param d
           ):
        ...

In [ ]:
DocmentTbl(_Test)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| a |  |  | description of param a |
| b | bool | True | description of param b |
| c | str | None |  |

You can also pass a method to be rendered as well:

In [ ]:
DocmentTbl(_Test.foo)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | int |  | description of param c |
| d | bool | True | description of param d |

In [ ]:
#| hide
_exp_res3 = """
|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| c | int |  | description of param c |
| d | bool | True | description of param d |
"""

test_eq(DocmentTbl(_Test.foo), _exp_res3)

In [ ]:
#| export
class DocmentList(_DocmentBase):
    def _fmt(self, nm, p):
        anno,default,doc = _maybe_nm(p.get('anno','')), p.get('default',empty), p.get('docment','')
        s = f'`{nm}{":" + anno if anno else ""}{"=" + _maybe_nm(default) if default != empty else ""}`'
        br = '\xa0'*3
        return f'- {s}' + (f'{br}*{doc}*' if doc else '')

    def _repr_markdown_(self): return '\n'.join(self._fmt(k,v) for k,v in self.dm.items())
    __repr__=__str__=_repr_markdown_

In [ ]:
DocmentList(_f)

- `a`
- `b:int`   *param b*
- `c:str=foo`   *param c*
- `return:str`   *Result of doing it*

In [ ]:
#| export
class DocmentText(_DocmentBase):
    def __init__(self, obj, maxline=110, docstring=True):
        super().__init__(obj)
        self.maxline,self.docstring = maxline,docstring

    def _fmt_param(self, nm, p):
        anno,default = p.get('anno',empty), p.get('default',empty)
        return nm + (f':{_maybe_nm(anno)}' if anno != empty else '') + (f'={repr(default)}' if default != empty else '')
    
    @property
    def _ret_str(self):
        ret = self.dm.get('return', {})
        anno = f"->{_maybe_nm(ret.get('anno',empty))}" if ret.get('anno',empty) != empty else ''
        doc = f" # {ret['docment']}" if ret.get('docment') else ''
        return f"){anno}:{doc}"
    
    @property
    def params(self): return [(self._fmt_param(k,v), v.get('docment','')) for k,v in self.dm.items() if k != 'return']

    def __str__(self):
        lines,curr = [],[]
        for fmt,doc in self.params:
            comment = f' # {doc}' if doc else ''
            if curr and len(', '.join(curr))+len(fmt)+len(comment)>self.maxline:
                lines.append(', '.join(curr) + ',')
                curr = []
            curr.append(fmt)
            if doc: lines.append(', '.join(curr) + ',' + comment); curr = []
        if curr: lines.append(', '.join(curr))
        body = '\n    '.join(lines)
        docstr = f'    "{self.obj.__doc__}"' if self.docstring and self.obj.__doc__ else ''
        return f"def {get_name(self.obj)}(\n    {body}\n{self._ret_str}\n{docstr}"
    
    __repr__ = __str__
    def _repr_markdown_(self): return f"```python\n{self}\n```"

In [ ]:
DocmentText(_f).params

[('a', None), ('b:int', 'param b'), ("c:str='foo'", 'param c')]

In [ ]:
DocmentText(_f)

```python
def _f(
    a, b:int, # param b
    c:str='foo', # param c
)->str: # Result of doing it
    "Do a thing"
```

In [ ]:
def _g(
    a, b:int, cccccccccccccccccccc:int, ccccccccdccccccccccc:int, cccccccccccecccccccc:int, cccccccfcccccccccc:int, ccccccccccccgccccc:int, # hi
    c:str='foo'
)->str:
    "Do a thing"

DocmentText(_g, maxline=80, docstring=False)

```python
def _g(
    a, b:int, cccccccccccccccccccc:int, ccccccccdccccccccccc:int,
    cccccccccccecccccccc:int, cccccccfcccccccccc:int, ccccccccccccgccccc:int, # hi
    c:str='foo'
)->str:

```

In [ ]:
DocmentText(partial(_g, 1), maxline=80, docstring=False)

```python
def _g[partial: 1](
    b:int, cccccccccccccccccccc:int, ccccccccdccccccccccc:int,
    cccccccccccecccccccc:int, cccccccfcccccccccc:int, ccccccccccccgccccc:int,
    c:str='foo'
)->str:

```

In [ ]:
#| export
def sig2str(func, maxline=110):
    "Generate function signature with docments as comments"
    return DocmentText(func, maxline=maxline, docstring=False)

In [ ]:
print(sig2str(_d))

def _d(
    c:int, # First
    b:str, # Second
    a:int=2, z:str='b', # Last
)->int: # Return an int



## Documentation For An Object

Render the signature as well as the `docments` to show complete documentation for an object.

In [ ]:
#| export
def _docstring(sym):
    npdoc = parse_docstring(sym)
    return '\n\n'.join([npdoc['Summary'], npdoc['Extended']]).strip()

In [ ]:
#| export
def _fullname(o):
    module,name = getattr(o, "__module__", None),qual_name(o)
    return name if module is None or module in ('__main__','builtins') else module + '.' + name

class ShowDocRenderer:
    def __init__(self, sym, name:str|None=None, title_level:int=3, maxline:int=110):
        "Show documentation for `sym`"
        sym = getattr(sym, '__wrapped__', sym)
        sym = getattr(sym, 'fget', None) or getattr(sym, 'fset', None) or sym
        store_attr()
        self.nm = name or qual_name(sym)
        self.isfunc = inspect.isfunction(sym)
        try: self.sig = signature(sym, eval_str=True)
        except (ValueError,TypeError): self.sig = None
        self.docs = _docstring(sym)
        self.dm = DocmentText(sym, maxline=maxline, docstring=False)
        self.fn = _fullname(sym)

    __repr__ = basic_repr()

In [ ]:
#| export
def _f_name(o): return f'<function {o.__name__}>' if isinstance(o, FunctionType) else None
def _fmt_anno(o): return inspect.formatannotation(o).strip("'").replace(' ','')

def _show_param(param):
    "Like `Parameter.__str__` except removes: quotes in annos, spaces, ids in reprs"
    kind,res,anno,default = param.kind,param._name,param._annotation,param._default
    kind = '*' if kind==inspect._VAR_POSITIONAL else '**' if kind==inspect._VAR_KEYWORD else ''
    res = kind+res
    if anno is not inspect._empty: res += f':{_f_name(anno) or _fmt_anno(anno)}'
    if default is not inspect._empty: res += f'={_f_name(default) or repr(default)}'
    return res

In [ ]:
#| hide
def _func(): pass
p = Parameter('foo', Parameter.POSITIONAL_OR_KEYWORD, default=_func, annotation='Callable')
test_eq(_show_param(p), 'foo:Callable=<function _func>')
p = p.replace(annotation=_func)
test_eq(_show_param(p), 'foo:<function _func>=<function _func>')

In [ ]:
#| export
def _ital_first(s:str):
    "Surround first line with * for markdown italics, preserving leading spaces"
    return re.sub(r'^(\s*)(.+)', r'\1*\2*', s, count=1)

In [ ]:
#| export
class MarkdownRenderer(ShowDocRenderer):
    "Markdown renderer for `show_doc`"
    def _repr_markdown_(self):
        doc = f'```python\n\n{self.dm}\n\n```'
        if self.docs: doc += f"\n\n{_ital_first(self.docs)}"
        return doc
    __repr__=__str__=_repr_markdown_

In [ ]:
def _f(a,
        b:int, #param b
        c:str='foo'  #param c
       )->str: # Result of doing it
    "Do a thing"
    ...

MarkdownRenderer(_f)

```python

def _f(
    a, b:int, # param b
    c:str='foo', # param c
)->str: # Result of doing it


```

*Do a thing*

In [ ]:
def f(a:int=0 # aa
): pass

@delegates(f)
def g(
    b:int|str, # bb
    **kwargs
): return kwargs

In [ ]:
MarkdownRenderer(g)

```python

def g(
    b:int | str, # bb
    a:int=0, # aa
):


```

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()